In [1]:
import numpy as np
import random

In [2]:
total_episodes = 50000        # Total episodes
learning_rate = 0.8           # Learning rate
max_steps = 500                # Max steps per episode
gamma = 0.95                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.005             # Exponential decay rate for exploration prob

In [3]:
#Action, State and reward
action_space = [0,1,2,3,4]#up,right,down,left,nothing
state_space_width=20 # S is start, T is tile, W is Wall, G is Goal
state_space_length=20
state_space = ['T','T','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','W',
               'T','T','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','G',
               'T','T','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','W','T','T',
               'T','T','T','T','T','T','T','W','W','T','W','T','T','T','T','T','W','T','T','T',
               'T','T','T','T','T','T','W','T','T','T','W','T','T','T','T','W','T','T','T','T',
               'T','T','T','T','T','T','W','T','W','T','W','T','T','T','W','T','T','T','T','T',
               'T','T','T','T','T','T','W','T','W','T','W','T','T','W','T','T','T','T','T','T',
               'T','W','T','T','T','T','W','T','W','T','W','T','W','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','T','W','T','T','W','T','T','T','T','T','T','T','T',
               'S','W','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'W','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T']
# for i in range(len(state_space)):
#     if not (state_space[i]=='S' or state_space[i]=='G'):
#         if(random.randint(0,8)==0):
#             state_space[i]='W'
#         else:
#             state_space[i]='T'
# print(len(state_space))
reward_space = []
for i in state_space:
    if(i=='S'):
        reward_space.append(-1)
    elif(i=='G'):
        reward_space.append(10)
    elif(i=='T'):
        reward_space.append(-1)
    elif(i=='W'):
        reward_space.append(-100)
print('  Reward matrix')
for i in range(len(reward_space)):
    print('{:4}'.format(reward_space[i]),end = ',')
    if(i%state_space_width==state_space_width-1):
        print()

  Reward matrix
  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,-100,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,-100,
  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,-100,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  10,
  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,-100,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,-100,  -1,  -1,
  -1,  -1,  -1,  -1,  -1,  -1,  -1,-100,-100,  -1,-100,  -1,  -1,  -1,  -1,  -1,-100,  -1,  -1,  -1,
  -1,  -1,  -1,  -1,  -1,  -1,-100,  -1,  -1,  -1,-100,  -1,  -1,  -1,  -1,-100,  -1,  -1,  -1,  -1,
  -1,  -1,  -1,  -1,  -1,  -1,-100,  -1,-100,  -1,-100,  -1,  -1,  -1,-100,  -1,  -1,  -1,  -1,  -1,
  -1,  -1,  -1,  -1,  -1,  -1,-100,  -1,-100,  -1,-100,  -1,  -1,-100,  -1,  -1,  -1,  -1,  -1,  -1,
  -1,-100,  -1,  -1,  -1,  -1,-100,  -1,-100,  -1,-100,  -1,-100,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
  -1,-100,  -1,  -1,  -1,  -1,  -1,  -1,-100,  -1,  -1,-100,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
  -1,-100,  -1,  -1,  -1,  -1,  -1,  -1,-100,  -1,  -1,  -1,  -1,  -1,  -1,

In [4]:
#Training
action_size = len(action_space)
state_size = len(state_space)

qtable = np.zeros((state_size, action_size))

rewards = []
#1. Initialize Q Table
for episode in range(total_episodes):
    # Reset the environment
    state = state_space.index('S')
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 2. Choose an action
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

        # Else doing a random choice --> exploration
        else:
            action = random.randint(0,action_space[-1])

        # Take the action (a) and observe the outcome state(s') and reward (r)
        state_x=state%state_space_width
        state_y=state//state_space_width
        if(action==0):
            if (state_y>0):
                state_y-=1
        elif(action==1):
            if (state_x<state_space_width-1):
                state_x+=1
        elif(action==2):
            if (state_y<state_space_length-1):
                state_y+=1
        elif(action==3):
            if (state_x>0):
                state_x-=1
        new_state=state_x+state_y*state_space_width
        reward=reward_space[new_state]
        if(state_space[new_state]=='G' or state_space[new_state]=='W'):
            done=True
        else:
            done=False
        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = (qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action]))
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print(qtable)

[[-12.93657872 -13.10870356 -12.93166909 -13.12986534 -12.89465781]
 [-12.85084641 -13.03403541 -13.15477778 -12.88008895 -12.85631759]
 [-13.10310606 -13.11746429 -13.17653308 -13.11443267 -13.08038304]
 ...
 [ -8.98014478  -9.07585842  -9.16219375  -9.16904451  -8.8485158 ]
 [ -8.93825301  -9.05513714  -8.87515148  -9.15183636  -8.80064597]
 [ -8.85386905  -8.80064597  -9.05334335  -9.13190567  -8.87288363]]


In [5]:
#Deploying
for episode in range(1):
    state = state_space.index('S')
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)
    steps_taken=[]
    steps_taken_xy=[]
    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        state_x=state%state_space_width
        state_y=state//state_space_width
        if(action==0):
            if (state_y>0):
                state_y-=1
        elif(action==1):
            if (state_x<state_space_width-1):
                state_x+=1
        elif(action==2):
            if (state_y<state_space_length-1):
                state_y+=1
        elif(action==3):
            if (state_x>0):
                state_x-=1
        new_state=state_x+state_y*state_space_width
        reward=reward_space[new_state]
        if(state_space[new_state]=='G' or state_space[new_state]=='W'):
            done=True
        else:
            done=False

        steps_taken_xy.append((state_x,state_y))
        steps_taken.append(new_state)
        if done:
            # print(steps_taken_xy)
            for i in range(len(state_space)):
                if(i not in steps_taken):
                    print('{:3}'.format(state_space[i]),end = ' ')
                else:
                    if(state_space[i] == 'G'):
                        print('{:3}'.format('A_G'),end = ' ')
                    else:
                        print('{:3}'.format('A'),end = ' ')
                if(i%state_space_width==state_space_width-1):
                    print()
            # We print the number of step it took.
            print("Number of steps", step)
            break
        state = new_state

****************************************************
EPISODE  0
T   T   T   T   T   T   T   T   W   T   T   T   T   T   T   T   T   T   T   W   
T   T   T   T   T   T   T   T   W   A   A   A   A   A   A   A   A   A   A   A_G 
T   T   T   T   T   T   T   T   W   A   T   T   T   T   T   T   T   W   T   T   
T   T   T   T   T   T   T   W   W   A   W   T   T   T   T   T   W   T   T   T   
T   T   T   T   T   T   W   A   A   A   W   T   T   T   T   W   T   T   T   T   
T   T   T   T   T   T   W   A   W   T   W   T   T   T   W   T   T   T   T   T   
A   A   A   A   A   A   W   A   W   T   W   T   T   W   T   T   T   T   T   T   
A   W   T   T   T   A   W   A   W   T   W   T   W   T   T   T   T   T   T   T   
A   W   T   T   T   A   A   A   W   T   T   W   T   T   T   T   T   T   T   T   
S   W   T   T   T   T   T   T   W   T   T   T   T   T   T   T   T   T   T   T   
T   W   T   T   T   T   T   T   T   W   T   T   T   T   T   T   T   T   T   T   
T   W   T   T   T   T   T   T   W   T   T   T